<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/notebook_util/render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title render images from 3D objects
#@markdown - The id of the run. Only used for specifying data_location and export_location
id = "172mqt5l" #@param {type:"string"}

#@markdown Enter 3D objects location.  
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
data_location = f"/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/{id}" #@param {type:"string"}

#@markdown Enter export location (folder/directory).   
#@markdown - For example via the file browser on the left to locate and right click to copy the path.)
#@markdown - file folder example: `/content/drive/My Drive/h/k` 
export_location = f"/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/{id}/img_folder" #@param {type:"string"}


#@markdown FID score
#@markdown - whether or not to calculate fid 
calculate_fid = False   #@param {type:"boolean"}
print("data_location:", data_location)
print("export_location:", export_location)
print("calculate_fid:", calculate_fid)


data_location: /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/172mqt5l
export_location: /content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/exportObjects/172mqt5l/img_folder
calculate_fid: False


In [3]:
#right click shared folder IRCMS_GAN_collaborative_database and "Add shortcut to Drive" to My drive
%cd drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-treegan/

#record paths to resources
data_path = "../../../../../My Drive/Hand-Tool-Data-Set/turbosquid_thingiverse_dataset/dataset_ply_out"
run_path = "./"

# !ls Research/Peter/Tree_3D_models_obj/obj_files/

/content/drive/.shortcut-targets-by-id/1ylB2p6N0qQ-G4OsBuwcZ9C0tsqVu9ww4/IRCMS_GAN_collaborative_database/Experiments/colab-treegan


In [4]:
!apt-get update
!pip install trimesh
!apt install -y xvfb
!pip install xvfbwrapper
!pip install wandb==0.9.7
output.clear()

In [5]:
import trimesh
import numpy as np
import torch
import wandb
import os
import io
from io import BytesIO
from xvfbwrapper import Xvfb
from PIL import Image

import zipfile
from trimesh.exchange.stl import HeaderError
from pathlib import Path

In [6]:
#render setup
vdisplay = Xvfb()
vdisplay.start()

def mesh2Image(voxelmesh):
    scene = voxelmesh.scene()
    try:
        png = scene.save_image(
            resolution=[600, 600],
        )
    except NoSuchDisplayException:
        print(
            "NoSuchDisplayException. Renderer not found! Please check configuation so trimesh scene.save_image() can run successfully"
        )
    png = io.BytesIO(png)
    image = Image.open(png)
    return image

def rotateMesh(voxelmesh, radians, axes):
    assert(len(radians) == len(axes))
    for i in range(len(axes)):
        ra = radians[i]
        ax = axes[i]
        voxelmesh = voxelmesh.apply_transform(
                trimesh.transformations.rotation_matrix(ra, ax)
            )
    return voxelmesh

def mesh2arrayCentered(mesh, voxel_size=1, array_length=64):
    # given array length 64, voxel size 2, then output array size is [128,128,128]
    array_size = np.ceil(
        np.array([array_length, array_length, array_length]) / voxel_size
    ).astype(int)
    vox_array = np.zeros(
        array_size, dtype=bool
    )  # tanh: voxel representation [-1,1], sigmoid: [0,1]
    # scale mesh extent to fit array_length
    max_length = np.max(np.array(mesh.extents))
    mesh = mesh.apply_transform(
        trimesh.transformations.scale_matrix((array_length - 1.5) / max_length)
    )  # now the extent is [array_length**3]
    # return mesh

    v = mesh.voxelized(voxel_size)  # max voxel array length = array_length / voxel_size

    # find indices in the v.matrix to center it in vox_array
    indices = ((array_size - v.matrix.shape) / 2).astype(int)
    vox_array[
        indices[0] : indices[0] + v.matrix.shape[0],
        indices[1] : indices[1] + v.matrix.shape[1],
        indices[2] : indices[2] + v.matrix.shape[2],
    ] = v.matrix

    return vox_array

In [7]:
success = []
failed = []

data_location = Path(data_location)
export_location = Path(export_location)
#create directory for export_location
if not export_location.exists():
    export_location.mkdir(parents=True)

file_ext = [".ply", ".stl", ".dae", ".obj", ".off", ".misc", ".gltf", ".assimp", ".threemf" ,".openctm" , ".xml_based", ".binvox", ".xyz"]


paths = [
            path
            for path in data_location.rglob("*.*")
            if path.suffix in file_ext and not "__MACOSX" in str(path)
        ]

for file_name in paths:
    try:
        m = trimesh.load(file_name, force='mesh')

        #   image viewpoints of meshes
        for x in [45, 0, -45]:
            for y in [0,90,180,270]:
                radians = [a/360 * 2 * np.pi for a in [y,x] ]
                axes = [(0,1,0), (1,0,0)]
                rot_mesh = rotateMesh(m.copy(), radians, axes)
                im = mesh2Image(rot_mesh)
                #construct save path
                if not (export_location/("x"+str(x)+"y"+str(y))).exists():
                    (export_location/("x"+str(x)+"y"+str(y))).mkdir(parents=True)

                path = export_location / ("x"+str(x)+"y"+str(y)) / (str(file_name.stem)+".png")
                im.save(path)

        success.append(file_name)
    except IndexError:
        failed.append(file_name)
        print(file_name+" failed")
print("number of files successfully rendered:",len(success))
print("number of files failed to render:",len(failed))

number of files successfully rendered: 20
number of files failed to render: 0
